Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

**Описание данных**

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Оглавление

[1. Загрузка данных](#glava_1)  
[2. Умножение матриц](#glava_2)  
[3. Алгоритм преобразования](#glava_3)  
[4. Проверка алгоритма](#glava_4)  
[Вывод](#conclusion)  

<a id="glava_1"></a>
## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import random

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

### Вывод

Таблица с данными состоит из 5000 строк и 5 столбцов.  

Пропуски отсутствуют.

<a id="glava_2"></a>
## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**  

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** ***Не изменится***

**Обоснование:** 

Умножим матрицу признаков на обратимую матрицу- P:  

$$
Z = XP  
$$
Формула вектора весов линейной регрессии будет выглядеть следующим образом:

$$
w1 = (Z^T Z)^{-1} Z^T y
$$

- $P$- обратимая матрица (матрица P с верхним индексом -1, произведение которой на P равно единичной матрице)   



Подставим вектор Z в формулу линейной регрессии и раскроем скобки:

$$
a1 = Z(Z^T Z)^{-1} Z^T y = Z(Z^T)^{-1} Z^{-1} Z^T y
$$

Запишем вместо вектора Z его значение:

$$
a1 = XP((XP)^T(XP))^{-1} (XP)^T y
$$
    
Выведем из скобок значение XP:

$$
a1 = XP(XP)^{-1}((XP)^T)^{-1}(XP)^T y
$$

$$
a1 = XPP^{-1}X^{-1}(P^T X^T)^{-1}P^T X^T y
$$
    
$$
a1 = XPP^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1} P^T X^T y
$$    
    
Учитывая, что умножение обратимой матрицы равно единичной матрице $PP^{-1} = E$ и $(P^T)^{-1} P^T = Е$, получим:

$$
a1 = X E X^{-1}(X^T)^{-1} E X^T y
$$

Если любую матрицу P умножить на единичную (или наоборот), получится эта же матрица P, следовательно единичные матрицы E можно сократить:

$$
a1 = X X^{-1}(X^T)^{-1}X^T y = X(X^T X)^{-1} X^T y = Xw
$$

$$
a1 = a
$$

Мы получили точно такое же уравнение линейной регрессии.

### Вывод

При умножении признаков на обратимую матрицу уравнение линейной регресси не изменяется, т.к. произведение квадратной матрицы на обратную (с индексом -1) равно единичной матрице.  
Если любую матрицу умножить на единичную (или наоборот), получится эта же матрица.

<a id="glava_3"></a>
## Алгоритм преобразования

**Алгоритм**

Для защиты данные клиентов предлагается использовать метод преобразования данных, основанный на умножении матрицы признаков на обратимую матрицу, сгенерированную случайно.

**Структура алгоритма:**  

1. Получение данных, матрица $X$ с n-количеством признаков (features);  
2. Генерация случайной матрицы $Z$ размера $n*n$;  
3. Проверка матрицы $Z$ на обратимость;
4. Умножение матрицы признаков $X$ на обратимую матрицу $Z$;
5. Вывод новых значений.

**Обоснование**

Учитывая выводы полученные во второй главе, можно утвержадть, что при умножении признаков на обратимую матрицу уравнение линейной регресси не изменяется.  

Основным критерием получения корректных значений является, подбор обратимой матрицы размерностью соответствующей количеству признаков для модели линейной регрессии.  

Квадратная матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель ($det (A)$) не равен нулю.  

Для сгенерированной случайной матрицы функцией numpy.random.normal(), вероятность получить необратимую матрицу близка к нулю.

<a id="glava_4"></a>
## Проверка алгоритма

Проведем обучение линейной регрессии и сравним метрику качества **R2 score** до преобразования и после.

Выделим целевое значение признаков из таблицы данных:

In [6]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

#### 1. Обучение модели до преобразования:

Разобъем данные на обучающую и тестовую выборку и обучим модель линейной регрессии:

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [8]:
model = LinearRegression(normalize=True)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
result_r2 = r2_score(target_test, predictions)
print('R2 score до преобразования:', result_r2)

R2 score до преобразования: 0.43522757127026657


Создадим функцию преобразования данных:

In [9]:
def code_features(features):
    n = features.shape[1]
    invertible_matrix = np.random.normal(0, 1, (n,n))
    try:
        check = np.linalg.det(invertible_matrix)
        while check == 0:
            invertible_matrix = np.random.normal(0, 1, (n,n))
            check = np.linalg.det(invertible_matrix)
        encryption_features = features @ invertible_matrix
        return encryption_features
    except:
        print("Что-то пошло не так...")

#### 2. Обучение модели после преобразования:

Применим функцию `code_features`, разобъем данные на обучающую и тестовую выборку и заново обучим модель линейной регрессии:

In [10]:
encrypt_features = code_features(features)

In [11]:
encrypt_features.head()

,0,1,2,3
0,70639.804009,75576.960732,45689.433450,62931.551683
1,54122.275467,57915.190779,35030.263101,48177.172853
2,29910.441065,32009.174750,19365.860973,26615.501532
3,59386.868175,63527.796433,38386.389622,52940.170340
4,37172.151526,39774.836551,24054.572434,33100.007342


Как мы видим, после выполнения функции, значения признаков **features** принимают случайные значения.

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    encrypt_features, target, test_size=0.25, random_state=12345)

In [13]:
model = LinearRegression(normalize=True)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
result_r2_after = r2_score(target_test, predictions)
print('R2 score после преобразования:', result_r2_after)

R2 score после преобразования: 0.43522757127035716


In [14]:
data= {'R2 score':[result_r2, result_r2_after]}
pd.DataFrame(data, index=('R2 score до преобразования', 'R2 score после преобразования'))

,R2 score
R2 score до преобразования,0.435228
R2 score после преобразования,0.435228


<a id="conclusion"></a>
## Вывод

Таблица с данными состоит из 5000 строк и 5 столбцов.

Выполнено доказательство, что при умножении признаков на обратимую матрицу уравнение линейной регресси не изменяется.  

Представлен алгоритм для защиты данных клиентов, основанный на умножении матрицы признаков на обратимую матрицу, сгенерированную случайно.  

Создана функция `code_features` для преобразования данных.

Обучены модели линейной регрессии, получены следующие значения метрики R2 score:


|Этапы| Значение R2 score |
| --- | --- |
| R2 score до преобразования | 0.435228 |
| R2 score после преобразования | 0.435228 |

Метрика качества не изменилась, следовательно преобразование выполненно корректно.  

Исходя из вышеизложенного можно сделать вывод, что используя обратимые матрицы с соответствующей размерностью, мы можем закодировать данные с сохранением точности результов обучения моделей.